In [1]:
import pandas as pd

req_cols = [' Destination Port',' Flow Duration',' Total Fwd Packets',' Total Backward Packets','Total Length of Fwd Packets',' Total Length of Bwd Packets',' Fwd Packet Length Max',' Fwd Packet Length Min',' Fwd Packet Length Mean',' Fwd Packet Length Std','Bwd Packet Length Max',' Bwd Packet Length Min',' Bwd Packet Length Mean',' Bwd Packet Length Std','Flow Bytes/s',' Flow Packets/s',' Flow IAT Mean',' Flow IAT Std',' Flow IAT Max',' Flow IAT Min','Fwd IAT Total',' Fwd IAT Mean',' Fwd IAT Std',' Fwd IAT Max',' Fwd IAT Min','Bwd IAT Total',' Bwd IAT Mean',' Bwd IAT Std',' Bwd IAT Max',' Bwd IAT Min','Fwd PSH Flags',' Bwd PSH Flags',' Fwd URG Flags',' Bwd URG Flags',' Fwd Header Length',' Bwd Header Length','Fwd Packets/s',' Bwd Packets/s',' Min Packet Length',' Max Packet Length',' Packet Length Mean',' Packet Length Std',' Packet Length Variance','FIN Flag Count',' SYN Flag Count',' RST Flag Count',' PSH Flag Count',' ACK Flag Count',' URG Flag Count',' CWE Flag Count',' ECE Flag Count',' Down/Up Ratio',' Average Packet Size',' Avg Fwd Segment Size',' Avg Bwd Segment Size',' Fwd Header Length','Fwd Avg Bytes/Bulk',' Fwd Avg Packets/Bulk',' Fwd Avg Bulk Rate',' Bwd Avg Bytes/Bulk',' Bwd Avg Packets/Bulk','Bwd Avg Bulk Rate','Subflow Fwd Packets',' Subflow Fwd Bytes',' Subflow Bwd Packets',' Subflow Bwd Bytes','Init_Win_bytes_forward',' Init_Win_bytes_backward',' act_data_pkt_fwd',' min_seg_size_forward','Active Mean',' Active Std',' Active Max',' Active Min','Idle Mean',' Idle Std',' Idle Max',' Idle Min',' Label']

fraction = 1

print('---------------------------------------------------------------------------------')
print('Loading Databases')
print('---------------------------------------------------------------------------------')
print('')

df0 = pd.read_csv ('cicids_db/Wednesday-workingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)
df1 = pd.read_csv ('cicids_db/Tuesday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)
df2 = pd.read_csv ('cicids_db/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)
df3 = pd.read_csv ('cicids_db/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)
df4 = pd.read_csv ('cicids_db/Monday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)
df5 = pd.read_csv ('cicids_db/Friday-WorkingHours-Morning.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)
df6 = pd.read_csv ('cicids_db/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)
df7 = pd.read_csv ('cicids_db/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)

frames = [df0, df1, df2, df3, df4, df5,df6, df7]

df = pd.concat(frames,ignore_index=True)
# df = df.sample(frac = 1)
# df = pd.concat(frames,ignore_index=True)
# df = df.sample(frac = fraction )
y = df.pop(' Label')
df = df.assign(Label = y)

---------------------------------------------------------------------------------
Loading Databases
---------------------------------------------------------------------------------



In [2]:
from collections import Counter
from sklearn.model_selection import train_test_split
import sklearn
print('---------------------------------------------------------------------------------')
print('Reducing Normal rows')
print('---------------------------------------------------------------------------------')
print('')

frac_normal = 0.2
split = 0.8

#filters
filtered_normal = df[df['Label'] == 'BENIGN']

#reduce
reduced_normal = filtered_normal.sample(frac=frac_normal)

#join
df = pd.concat([df[df['Label'] != 'BENIGN'], reduced_normal])

''' ---------------------------------------------------------------'''
df_max_scaled = df.copy()


y = df_max_scaled['Label'].replace({'DoS GoldenEye': 'Dos/Ddos', 'DoS Hulk': 'Dos/Ddos', 'DoS Slowhttptest': 'Dos/Ddos', 'DoS slowloris': 'Dos/Ddos', 'Heartbleed': 'Dos/Ddos', 'DDoS': 'Dos/Ddos','FTP-Patator': 'Brute Force', 'SSH-Patator': 'Brute Force','Web Attack - Brute Force': 'Web Attack', 'Web Attack - Sql Injection': 'Web Attack', 'Web Attack - XSS': 'Web Attack'})

df_max_scaled.pop('Label')


print('---------------------------------------------------------------------------------')
print('Normalizing database')
print('---------------------------------------------------------------------------------')
print('')

df_max_scaled
for col in df_max_scaled.columns:
    t = abs(df_max_scaled[col].max())
    df_max_scaled[col] = df_max_scaled[col]/t
df_max_scaled
df = df_max_scaled.assign( Label = y)
#df
df = df.fillna(0)

y = df.pop('Label')
X = df
df = df.assign(Label = y)

counter = Counter(y)
print(counter)


X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=split,random_state=42)
df = X.assign( Label = y)
# df[['bp_before','bp_after']].describe()

---------------------------------------------------------------------------------
Reducing Normal rows
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Normalizing database
---------------------------------------------------------------------------------

Counter({'BENIGN': 454619, 'Dos/Ddos': 380699, 'PortScan': 158930, 'Brute Force': 13835, 'Web Attack': 2180, 'Bot': 1966, 'Infiltration': 36})


In [3]:
#Model Parameters RF
from sklearn.ensemble import RandomForestClassifier
max_depth = 5
n_estimators = 5
min_samples_split = 2

print('---------------------------------------------------------------------------------')
print('Defining the RF model')
print('---------------------------------------------------------------------------------')
print('')

rf = RandomForestClassifier(max_depth = max_depth,  n_estimators = n_estimators, min_samples_split = min_samples_split, n_jobs = -1)

# model = clf.fit(X, y)

---------------------------------------------------------------------------------
Defining the RF model
---------------------------------------------------------------------------------



In [4]:
#Model Parameters SVM
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier

max_iter=10
loss='hinge'
gamma=0.1


rbf_feature = RBFSampler(gamma=gamma, random_state=1)
X_features = rbf_feature.fit_transform(X_train)
svm = SGDClassifier(max_iter=max_iter,loss=loss)
# clf.fit(X_features, y_train)
# clf.score(X_features, y_train)


In [5]:
#Model Parameters ADA
from sklearn.ensemble import AdaBoostClassifier
n_estimators=100
learning_rate=0.5

ada = AdaBoostClassifier(n_estimators=n_estimators,learning_rate=learning_rate)


In [32]:

# #Model Parameters DNN
# import tensorflow as tf
# dropout_rate = 0.01
# nodes = 70
# out_layer = 7
# optimizer='adam'
# loss='sparse_categorical_crossentropy'
# epochs=5
# batch_size=2*256


# print('---------------------------------------------------------------------------------')
# print('Defining the DNN model')
# print('---------------------------------------------------------------------------------')
# print('')


# num_columns = X_train.shape[1]

# dnn = tf.keras.Sequential()

# # Input layer
# dnn.add(tf.keras.Input(shape=(num_columns,)))

# # Dense layers with dropout
# dnn.add(tf.keras.layers.Dense(nodes))
# dnn.add(tf.keras.layers.Dropout(dropout_rate))

# dnn.add(tf.keras.layers.Dense(nodes))
# dnn.add(tf.keras.layers.Dropout(dropout_rate))

# dnn.add(tf.keras.layers.Dense(nodes))
# dnn.add(tf.keras.layers.Dropout(dropout_rate))

# dnn.add(tf.keras.layers.Dense(nodes))
# dnn.add(tf.keras.layers.Dropout(dropout_rate))

# dnn.add(tf.keras.layers.Dense(nodes))
# dnn.add(tf.keras.layers.Dropout(dropout_rate))

# # Output layer
# dnn.add(tf.keras.layers.Dense(out_layer))



# dnn.compile(optimizer=optimizer, loss=loss)

# dnn.summary()

import tensorflow as tf
from scikeras.wrappers import KerasClassifier, KerasRegressor

def getModel(optimizer):


    num_columns = X_train.shape[1]

    dnn = tf.keras.Sequential()

    # Input layer
    dnn.add(tf.keras.Input(shape=(num_columns,)))

    # Dense layers with dropout
    dnn.add(tf.keras.layers.Dense(nodes))
    dnn.add(tf.keras.layers.Dropout(dropout_rate))

    dnn.add(tf.keras.layers.Dense(nodes))
    dnn.add(tf.keras.layers.Dropout(dropout_rate))

    dnn.add(tf.keras.layers.Dense(nodes))
    dnn.add(tf.keras.layers.Dropout(dropout_rate))

    dnn.add(tf.keras.layers.Dense(nodes))
    dnn.add(tf.keras.layers.Dropout(dropout_rate))

    dnn.add(tf.keras.layers.Dense(nodes))
    dnn.add(tf.keras.layers.Dropout(dropout_rate))

    # Output layer
    dnn.add(tf.keras.layers.Dense(out_layer))



    dnn.compile(optimizer=optimizer, loss=loss)

    return dnn


#Model Parameters DNN
dropout_rate = 0.01
nodes = 70
out_layer = 7
optimizer='adam'
loss='sparse_categorical_crossentropy'
epochs=5
batch_size=2*256
# optimizer = ['Adam']
epochs = [5]


param_grid = dict(epochs=epochs, optimizer=optimizer)
Kmodel = KerasClassifier(build_fn=getModel, verbose=1)

In [7]:
#Model Parameters KNN
from sklearn.neighbors import KNeighborsClassifier
n_neighbors=5

knn_clf=KNeighborsClassifier(n_neighbors=n_neighbors)

In [9]:
#Model Parameters LGBM
from lightgbm import LGBMClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
n_split= 3
n_repeat= 15

lgbm = LGBMClassifier()
# cv = RepeatedStratifiedKFold(n_splits=n_split, n_repeats=n_repeat)
# n_scores = cross_val_score(lgbm, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# fit the model on the whole dataset
# lgbm = LGBMClassifier()



In [10]:
#Model Parameters MLP
from sklearn.neural_network import MLPClassifier
max_iter=70
# MLP = MLPClassifier(random_state=1, max_iter=max_iter).fit(X_train, y_train)
MLP = MLPClassifier(random_state=42, max_iter=max_iter)

In [33]:
from scipy.stats import wilcoxon
from sklearn.datasets import load_iris
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold

# Prepare models and select your CV method
# model1 = ExtraTreesClassifier()
# model2 = RandomForestClassifier()
model1 = svm
model2 = rf
model3 = ada
model4 = Kmodel
model5 = lgbm
model6 = MLP
model7 = knn_clf
kf = KFold(n_splits=20, random_state=42)

# Extract results for each model on the same folds
# results_model1 = cross_val_score(model1, X, y, cv=kf)
# results_model2 = cross_val_score(model2, X, y, cv=kf)
# results_model3 = cross_val_score(model3, X, y, cv=kf)
# results_model4 = cross_val_score(model4, X, y, cv=kf)
# results_model5 = cross_val_score(model5, X, y, cv=kf)
# results_model6 = cross_val_score(model6, X, y, cv=kf)
# results_model7 = cross_val_score(model7, X, y, cv=kf)


/home/oarreche@ads.iu.edu/anaconda3/envs/HITL/lib/python3.6/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [13]:

# Extract results for each model on the same folds
results_model1 = cross_val_score(model1, X, y, cv=kf) #SVM
results_model2 = cross_val_score(model2, X, y, cv=kf) # RF

In [14]:
# Calculate p value
#SVM and RF
stat, p = wilcoxon(results_model1, results_model2, zero_method='zsplit'); p

0.000133514404296875

In [15]:
#ADA
results_model3 = cross_val_score(model3, X, y, cv=kf)

In [16]:
#SVM and ADA
stat, p = wilcoxon(results_model1, results_model3,  zero_method='zsplit'); p

0.08969497680664062

In [17]:
#RF and ADA
stat, p = wilcoxon(results_model2, results_model3,  zero_method='zsplit'); p

3.814697265625e-06

In [34]:
#DNN
results_model4 = cross_val_score(model4, X, y, cv=kf)

/home/oarreche@ads.iu.edu/anaconda3/envs/HITL/lib/python3.6/site-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/home/oarreche@ads.iu.edu/anaconda3/envs/HITL/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/oarreche@ads.iu.edu/anaconda3/envs/HITL/lib/python3.6/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/oarreche@ads.iu.edu/anaconda3/envs/HITL/lib/python3.6/site-packages/scikeras/wrappers.py", line 1465, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/home/oarrech

In [35]:
#SVM and DNN
stat, p = wilcoxon(results_model1, results_model4,  zero_method='zsplit'); p

1.9073486328125e-06

In [36]:
#RF and DNN
stat, p = wilcoxon(results_model2, results_model4,  zero_method='zsplit'); p

1.9073486328125e-06

In [37]:
#ADA and DNN
stat, p = wilcoxon(results_model3, results_model4,  zero_method='zsplit'); p

1.9073486328125e-06

In [19]:
#LGBM
results_model5 = cross_val_score(model5, X, y, cv=kf)

In [20]:
#SVM and LGBM
stat, p = wilcoxon(results_model1, results_model5,  zero_method='zsplit'); p

0.017181396484375

In [21]:
#RF and LGBM
stat, p = wilcoxon(results_model2, results_model5,  zero_method='zsplit'); p

0.9272785186767578

In [22]:
#ADA and LGBM
stat, p = wilcoxon(results_model3, results_model5,  zero_method='zsplit'); p

3.814697265625e-06

In [38]:
#DNN and LGBM
stat, p = wilcoxon(results_model4, results_model5,  zero_method='zsplit'); p

1.9073486328125e-06

In [23]:
# MLP
results_model6 = cross_val_score(model6, X, y, cv=kf)


/home/oarreche@ads.iu.edu/anaconda3/envs/HITL/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (70) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/oarreche@ads.iu.edu/anaconda3/envs/HITL/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (70) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/oarreche@ads.iu.edu/anaconda3/envs/HITL/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (70) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/oarreche@ads.iu.edu/anaconda3/envs/HITL/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: 

In [24]:
#SVM and MLP
stat, p = wilcoxon(results_model1, results_model6,  zero_method='zsplit'); p

9.5367431640625e-06

In [25]:
#RF and MLP
stat, p = wilcoxon(results_model2, results_model6,  zero_method='zsplit'); p

0.069580078125

In [26]:
#ADA and MLP
stat, p = wilcoxon(results_model3, results_model6,  zero_method='zsplit'); p

1.9073486328125e-06

In [39]:
#DNN and MLP
stat, p = wilcoxon(results_model4, results_model6,  zero_method='zsplit'); p

1.9073486328125e-06

In [27]:
#LGBM and MLP
stat, p = wilcoxon(results_model5, results_model6,  zero_method='zsplit'); p

0.03276824951171875

In [40]:
# KNN
results_model7 = cross_val_score(model7, X, y, cv=kf)

In [46]:
#SVM and KNN
stat, p = wilcoxon(results_model1, results_model7,  zero_method='zsplit'); p

1.9073486328125e-06

In [41]:
#RF and KNN
stat, p = wilcoxon(results_model2, results_model7,  zero_method='zsplit'); p

0.0003948211669921875

In [42]:
#ADA and KNN
stat, p = wilcoxon(results_model3, results_model7,  zero_method='zsplit'); p

3.814697265625e-06

In [43]:
#DNN and KNN
stat, p = wilcoxon(results_model4, results_model7,  zero_method='zsplit'); p

1.9073486328125e-06

In [44]:
#LGBM and KNN
stat, p = wilcoxon(results_model5, results_model7,  zero_method='zsplit'); p

0.0004825592041015625

In [45]:
#MLP and KNN
stat, p = wilcoxon(results_model6, results_model7,  zero_method='zsplit'); p

0.009435653686523438